# WeatherPy

## Analysis

In [9]:
import pandas as pd
import json
import requests as rq
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from citipy import citipy

In [10]:
city = citipy.nearest_city(22.99, 120.21)
city = city.city_name
city

'tainan'

## Generate Cities List

In [18]:
weather_df = pd.DataFrame(columns= ['City'])
for i in range(550):
    lat = np.random.uniform(low= -90, high=90)
    long = np.random.uniform(low=-180, high=180)
    city = citipy.nearest_city(lat, long)
    city = city.city_name
    weather_df = weather_df.append({'City':city}, ignore_index=True)
    
weather_df.head()

,City
0,oktyabrskiy
1,rikitea
2,ojinaga
3,saldanha
4,bluff


## Perform API Calls

In [ ]:
url = 'api.openweathermap.org/data/2.5/weather'
parameters = {'q':city}

Latitude = []
Temperature = []
Cloudness = []
WindSpeed = []



## Latitude vs Temperature Plot

## Latitude vs Humidity Plot

## Latitude vs Cloudness Plot

## Latitude vs Wind Speed Plot

In [ ]:
#write to csv